In [ ]:
from github import Github
import pandas as pd 
import itertools
import json
import os

##### Load personal token here to authenticate.

In [ ]:
with open(os.path.expanduser('~') + '/.github_shell_token', 'r') as token_file:
    access_token = token_file.read().strip()
if access_token == None:
    error("Error: no access token found or provided")
g = Github(access_token)

In [ ]:
g.rate_limiting

In [ ]:
org=g.get_organization("Carpentries-Incubator")

#### Total number of repositories.

In [ ]:
repo = g.get_organization("carpentries-incubator").get_repos()
repo.totalCount

#### Creating a page_num variable to hold the valid page numbers to iterate the repo list.

In [ ]:

page_num = 0
for i in range(0,10):
    if len(repo.get_page(i))==0:
        page_num = i
        break
    else:
        pass

In [ ]:
repo_list = list()
for i in range(0,page_num):
    repo_list.append(repo.get_page(i))

#### Final repo list created from repo list

final_repo_list = list((itertools.chain.from_iterable(repo_list)))

#### Repository _list_with_lesson & Repository _list_without_lesson.

In [ ]:
list_without_lesson=list()
list_with_lesson=list()
for repo in final_repo_list:
    tags=repo.get_topics()
    if "lesson" in tags:
        list_with_lesson.append(repo)
    else:
        list_without_lesson.append(repo)
print("List of Repository with lesson:")   
for repo in list_with_lesson:
    print(repo)
print("\nList of Repository without lesson\n")
for repo in list_without_lesson:
    print(repo)

In [ ]:
# Truncating the for testing purpose.

# list_with_lesson=list_with_lesson[:5]

In [ ]:
list_with_lesson

#### Create Functions to determine & implement parameters describing project activities.in particular

* number of open issues
* number of closed issues
* number of open PRs-0
* number of closed PRs
* number of contributors
* number of starts

In [ ]:
# Declaring a function to count open_issue

def number_of_open_issues(repo):
    count_all_open_issue=0
    open_issues=repo.get_issues(state='open')
    for issue in open_issues:
        count_all_open_issue+=1
    return count_all_open_issue


In [ ]:
# Declaring a function to count closed_issue

def number_of_closed_issues(repo):
    count_all_closed_issue=0
    closed_issues=repo.get_issues(state='closed')
    for issue in closed_issues:
        count_all_closed_issue+=1
    return count_all_closed_issue

In [ ]:
# Declaring a function to count stargazers

def stargazers(repo):
    stars=repo.stargazers_count
    return stars


In [ ]:
# Declaring a function to count closed_pr

def number_of_closed_PR(repo):
    count_all_closed_PR=0
    pulls=repo.get_pulls(state='closed')
    for pr in pulls:
        count_all_closed_PR+=1
    return count_all_closed_PR
    

In [ ]:
# Declaring a function to count closed_pr

def number_of_open_PR(repo):
    count_all_open_PR=0
    pulls=repo.get_pulls(state='open')
    for pr in pulls:
        count_all_open_PR+=1
    return count_all_open_PR

In [ ]:
# Creating a total count of all categories (Issue_count, Pr_count, Stagazers_count & contributors_count)

def get_final_values(list_with_lesson):
    total_closed_Issue_count=0
    total_open_Issue_count=0
    starCount=0
    total_OpenPR_Request=0
    total_ClosedPR_Request=0
    for repo in list_with_lesson:
        
        # closed issues_count
        closedIssues = number_of_closed_issues(repo)
        total_closed_Issue_count += closedIssues
        
        # open issues_count
        openIssues = number_of_open_issues(repo)
        total_open_Issue_count+=openIssues
        
        # Stargazers_count
        stars = stargazers(repo)
        starCount+=stars
        
        # open pr_request_count
        openPR = number_of_open_PR(repo)
        total_OpenPR_Request+=openPR
        
        # closed pr_request_count
        closedPR_Request= number_of_closed_PR(repo)
        total_ClosedPR_Request+=closedPR_Request

    return {'total_closed_Issue_count':total_closed_Issue_count,
            'total_open_Issue_count':total_open_Issue_count,
            'starCount':starCount,
            'total_OpenPR_request': total_OpenPR_Request,
            'total_ClosedPR_request':total_ClosedPR_Request}


# Calling the function

final_values = get_final_values(list_with_lesson)
print(final_values)

#### Dashboard that presents the description of all lessons in the Carpentries incubator.

* lesson title
* repository
* Life Cycle Stage
* Tags
* Last updated

In [ ]:
#### Declaring a function to get_lesson_title,last_updated_date,repsoitory_name,Life_cycle_stage of lesson ,
 ##  tags in each lesson.

import datetime
def lesson_info(repo):
    
    """
    This function is giving overview of each repsoitory.
    Input:- Taking repo as a parameter that contains all the repositories information of Carpentries Incubator.
    Output:- Returns a dataframe 
    
    """
    
    thisdict={}
    version=""
    
    # To get the updated date & time.
    date_latest=datetime.datetime(2005, 6, 1)
    
    # To get the repositories name
    name=repo.name
    
    # Calling get_topics method to get the topics of each repositories
    tags=repo.get_topics()
    
    #Calling the description object to get the title of lessons
    
    title=repo.description
    
    help_wanted=0
    if "alpha" in tags:
        version="alpha"
    if "pre-alpha" in tags:
        version="pre-alpha"
    if "beta" in tags:
        version="beta"
    if "stable" in tags:
        version="stable"
        
    # Calling the get_issues method to get all the issues to retreive the labels of repsositories.
    
    open_issues=repo.get_issues(state='open')
    for issue in open_issues:
        labels=issue.get_labels()
        for label in labels:
            if label.name=="help wanted":
                help_wanted+=1
                
    # Calling the get_commits method to get all the commits to retrieve last updated date & time of the repsoitories.
    
    commits=repo.get_commits()
    for commit in commits:
        time = datetime.datetime.strptime(commit.last_modified,'%a, %d %b %Y %H:%M:%S %Z')
        if (time > date_latest):
            date_latest = time
            
    # Putting the date into a dictionary.
  
    thisdict["repo_name"]=name
    thisdict["repo_tags"]=tags
    thisdict["repo_description"]=title
    thisdict["repo_datetime"]=date_latest
    thisdict["repo_version"]=version
    thisdict["repo_label"]=help_wanted
    return thisdict

# Assigning a dictionary to list_of_topic

list_of_topic=[]
for repo in list_with_lesson:

# Calling the function.

    temp_repo_info=lesson_info(repo)
    list_of_topic.append(temp_repo_info)


In [ ]:
df=pd.json_normalize(list_of_topic)
pd.set_option('display.max_rows', None)
df

#### Declaring a function to get the overview of repositories.

*  Total number of lessons in the incubator
*  how many lessons at each state
*  number of lessons with the "help wanted" tag

In [ ]:
# Declaring a function to get number of lesson in the Incubator,count_of_lesson at each state and number_help_wanted_tag.

def lesson_overview(list_of_topic):
    
    """
    This function gives the overview of each repsoistory.
    Input:- It takes list_of_topics which contains a distionary holds repo_name, repo_tags, last updated date & time,
            repo_title, lesson life cycle stage ,repo labels.
    Output:- Returns a dictionary of final Value that contains count of lesson at pre-alpha, alpha,beta and stable
             and count of lessons that have help_wanted tag. Further count of lessons updated in the last 30 days.
    
    """
    
    count_all_alpha_state=0
    count_all_pre_alpha_state=0
    count_all_beta_state=0
    count_all_stable_state=0
    total_lesson_count=0
    totallesson_help_wanted=0
    totallesson_updated_last_30_days=0
    today = datetime.datetime.now()
    previous_month = today.replace(month = datetime.datetime.now().month-2)
    
    for topic in list_of_topic:
        total_lesson_count+=1
        
        if topic["repo_version"]=="alpha":
            count_all_alpha_state+=1
        
        if topic["repo_version"]=="pre-alpha":
            count_all_pre_alpha_state+=1
        
        if topic["repo_version"]=="beta":
            count_all_beta_state+=1
        
        if topic["repo_version"]=="stable":
            count_all_stable_state+=1
        
        if topic["repo_label"]!=0:
            totallesson_help_wanted+=1
        
        if topic["repo_datetime"]>previous_month:
            totallesson_updated_last_30_days+=1
        
    return {'total_lesson_count':total_lesson_count,
            'count_all_alpha_state':count_all_alpha_state ,
            'count_all_pre_alpha_state':count_all_pre_alpha_state,
            'count_all_beta_state':count_all_beta_state,
            'count_all_stable_state':count_all_stable_state,
            'totallesson_help_wanted':totallesson_help_wanted, 
            'totallesson_updated_last_30_days':totallesson_updated_last_30_days}


finalvalues=lesson_overview(list_of_topic)

print(finalvalues)